In [1]:
import pandas as pd, geopandas as gpd
import calendar, time
import json

Load fatalities dataset
Load cities shapefile

In [2]:
cities = gpd.read_file("../shp/Russian cities.shp",crs={'init' :'epsg:4326'})
cities = cities[["NAME", "geometry"]]
source = "../geolocated_datasets/1686574174R-geolocated-@poisk_in_ua.json"
with open(source, encoding = "utf-8-sig") as source_file:
    source_gdf = gpd.read_file(source_file, driver = "GeoJSON")

Convert both datasest to ESPG:3857 to allow use of meter system

In [3]:
source_gdf = source_gdf.to_crs(3857)

Create buffer of 10kms

In [4]:
buffer_distance = 10 * 1000
cities['lon'] = cities["geometry"].x
cities['lat'] = cities["geometry"].y
cities_buffer = cities.to_crs(3857)

cities_buffer.geometry = cities_buffer.buffer(buffer_distance)

Spatial join on buffers overlapping with fatalities dataset

In [5]:
pointsOnCity = gpd.sjoin(source_gdf, cities_buffer, how="right")
pointsOnCity['count']=1
fatalities_per_city = pointsOnCity.groupby('NAME').agg({'geometry':'first', 'lat':'first', 'lon':'first', 'count':'sum'}).reset_index()
fatalities_per_city.set_geometry("geometry", crs = "EPSG:4326", inplace = True)

In [6]:
fatalities_per_city

,NAME,geometry,lat,lon,count
0,Abakan,"POLYGON ((10189611.05823 7114236.22653, 101895...",53.703685,91.445002,1
1,Aldan,"POLYGON ((13968283.04568 8095069.80672, 139682...",58.602998,125.389390,1
2,Angarsk,"POLYGON ((11578321.79493 6902022.32842, 115782...",52.560007,103.920003,1
3,Archangel,"POLYGON ((4523449.65590 9497294.04510, 4523401...",64.574959,40.545008,1
4,Astrakhan,"POLYGON ((5359456.90556 5836398.61967, 5359408...",46.348655,48.054989,1
...,...,...,...,...,...
72,Yakutsk,"POLYGON ((14452035.94144 8867436.89558, 144519...",62.034959,129.735016,1
73,Yaroslavl,"POLYGON ((4448309.26678 7887907.92460, 4448261...",57.619983,39.870010,1
74,Yekaterinburg,"POLYGON ((6755740.03930 7729923.79958, 6755691...",56.851976,60.598014,1
75,Yuzhno Sakhalinsk,"POLYGON ((15899745.28469 5936358.87577, 158996...",46.964974,142.740011,1


Export dataset to geolocated_dataset folder

In [7]:
timestamp = calendar.timegm(time.gmtime())
fatalities_per_city_fn = f"../geolocated_datasets/{timestamp}-geolocated_fatalities_per_city.json"
output_file = open(fatalities_per_city_fn, "w", encoding="utf-8-sig")
contents = fatalities_per_city.to_json()
output_file.write(contents)

215340